# 부동산 분석
- 데이터: https://data.seoul.go.kr/dataList/OA-21275/S/1/datasetView.do
- 

In [17]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('data/서울시 부동산 실거래가 정보_2023.csv', encoding='euc-kr')

In [42]:
def make_building_code(row):
    use_columns = ['자치구명', '법정동명', '본번', '부번']
    is_nan = False
    for col in use_columns:
        is_nan = is_nan | pd.isna(row[col])
    if is_nan:
        return np.nan
    
    building_code = []
    for col in use_columns:
        col_code = row[col]
        if not isinstance(col_code, str):
            col_code = str(int(col_code))
        building_code.append(col_code)
    building_code = '-'.join(building_code)  # ['a', 'b', 'c'] --> 'a-b-c'
    return building_code

In [49]:
df['계약일']

0        20230913
1        20230913
2        20230913
3        20230913
4        20230913
           ...   
49129    20221025
49130    20220530
49131    20220523
49132    20220315
49133    20200608
Name: 계약일, Length: 49134, dtype: int64

In [50]:
start_date = 20230801
th_date = 20230901
end_date = 20230914

In [54]:
df_prior = df[(df['계약일'] >= start_date) & (df['계약일'] < th_date)]

In [55]:
df_posterior = df[(df['계약일'] >= 20230901) & (df['계약일'] < end_date)]

In [61]:
df_prior_price = df_prior.groupby('건물코드').mean()['물건금액(만원)']

In [62]:
df_posterior_price = df_posterior.groupby('건물코드').mean()['물건금액(만원)']

In [67]:
df_prior_price = pd.DataFrame(df_prior_price).reset_index()
df_prior_price.columns = ['건물코드', '물건금액(만원)_prior']

In [68]:
df_posterior_price = pd.DataFrame(df_posterior_price).reset_index()
df_posterior_price.columns = ['건물코드', '물건금액(만원)_posterior']

In [76]:
df_merge = df_prior_price.merge(df_posterior_price, on='건물코드')

In [79]:
df_merge['ratio'] = (df_merge['물건금액(만원)_posterior']/df_merge['물건금액(만원)_prior'] - 1)*100

In [96]:
df_merge['주소_구'] = df_merge.apply(lambda x:x['건물코드'].split('-')[0], axis=1)
df_merge['주소_동'] = df_merge.apply(lambda x:' '.join(x['건물코드'].split('-')[:2]), axis=1)

In [95]:
df_merge.groupby('주소').mean()['ratio'].sort_values()

주소
영등포구 대림동      -32.275132
영등포구 당산동6가    -27.272727
강동구 상일동       -17.961903
노원구 중계동       -16.908930
은평구 수색동       -15.447805
                 ...    
용산구 이태원동       43.020073
금천구 독산동       110.617267
용산구 문배동       125.423729
종로구 사직동       275.641026
종로구 내수동       285.520536
Name: ratio, Length: 132, dtype: float64

In [97]:
df_merge.groupby('주소_구').mean()['ratio'].sort_values()

주소_구
중랑구      -7.860449
구로구      -5.227198
동대문구     -5.017144
은평구      -4.451871
관악구      -3.096512
노원구      -2.205352
양천구      -1.691259
성북구      -0.539158
영등포구      0.635759
송파구       0.652155
강서구       0.885938
광진구       1.449003
동작구       1.755353
중구        2.578430
서초구       3.402614
강동구       3.963062
도봉구       4.926320
마포구       6.288729
강남구       7.597298
서대문구     14.999968
성동구      17.907724
강북구      22.207027
용산구      42.396614
금천구      44.444521
종로구     171.398770
Name: ratio, dtype: float64

In [100]:
df_merge[df_merge['주소_구'] == '종로구']

,건물코드,물건금액(만원)_prior,물건금액(만원)_posterior,ratio,주소,주소_구,주소_동
298,종로구-내수동-72-0,31400.0,208000.0,562.420382,종로구 내수동,종로구,종로구 내수동
299,종로구-내수동-75-0,29000.0,31500.0,8.620690,종로구 내수동,종로구,종로구 내수동
300,종로구-무악동-82-0,96726.0,117900.0,21.890702,종로구 무악동,종로구,종로구 무악동
301,종로구-사직동-9-0,39000.0,146500.0,275.641026,종로구 사직동,종로구,종로구 사직동
302,종로구-숭인동-2-1,95000.0,84000.0,-11.578947,종로구 숭인동,종로구,종로구 숭인동


In [106]:
df[df['건물코드'] == '종로구-내수동-72-0'].iloc[1]

접수연도                           2023
자치구코드                         11110
자치구명                            종로구
법정동코드                         11800
법정동명                            내수동
지번구분                            1.0
지번구분명                            대지
본번                             72.0
부번                              0.0
건물명                     경희궁의 아침 3단지
계약일                        20230807
물건금액(만원)                      31400
건물면적(㎡)                        37.5
토지면적(㎡)                       48.95
층                              16.0
권리구분                            NaN
취소일                             NaN
건축년도                         2004.0
건물용도                           오피스텔
신고구분                           중개거래
신고한 개업공인중개사 시군구명    서울 영등포구, 서울 종로구
건물코드                   종로구-내수동-72-0
Name: 5669, dtype: object

In [105]:
df[df['건물코드'] == '종로구-내수동-72-0'].iloc[0]

접수연도                          2023
자치구코드                        11110
자치구명                           종로구
법정동코드                        11800
법정동명                           내수동
지번구분                           1.0
지번구분명                           대지
본번                            72.0
부번                             0.0
건물명                      경희궁의아침3단지
계약일                       20230909
물건금액(만원)                    208000
건물면적(㎡)                     150.48
토지면적(㎡)                        0.0
층                              9.0
권리구분                           NaN
취소일                            NaN
건축년도                        2004.0
건물용도                           아파트
신고구분                          중개거래
신고한 개업공인중개사 시군구명    서울 용산구, 서울 종로구
건물코드                  종로구-내수동-72-0
Name: 143, dtype: object

In [86]:
df_merge['건물코드'].str.split('-')

0      [강남구, 개포동, 1231, 21]
1       [강남구, 대치동, 1027, 0]
2        [강남구, 대치동, 316, 0]
3        [강남구, 도곡동, 467, 6]
4         [강남구, 도곡동, 91, 5]
               ...         
313      [중랑구, 신내동, 650, 0]
314      [중랑구, 신내동, 661, 0]
315      [중랑구, 신내동, 785, 0]
316      [중랑구, 신내동, 816, 0]
317      [중랑구, 중화동, 450, 0]
Name: 건물코드, Length: 318, dtype: object

In [71]:
pd.merge([df_prior_price, df_posterior_price], on='건물코드')

TypeError: merge() missing 1 required positional argument: 'right'

In [69]:
df_posterior_price

,건물코드,물건금액(만원)_posterior
0,강남구-개포동-1172-3,30500.0
1,강남구-개포동-1231-21,27500.0
2,강남구-개포동-13-3,19800.0
3,강남구-논현동-172-2,35000.0
4,강남구-논현동-22-0,87000.0
...,...,...
776,중랑구-중화동-309-106,25250.0
777,중랑구-중화동-322-34,30600.0
778,중랑구-중화동-330-100,36500.0
779,중랑구-중화동-450-0,54000.0
